In [1]:
import os
from smosaic import collection_query, collection_get_data, mosaic

In [2]:
path_dir = os.path.abspath("")

In [3]:
collection=collection_query(
    collection="AMZ1-WFI-L4-SR-1",
    start_date="2025-01-01",
    end_date="2025-02-28",
    bbox="-74.8828,-34.5970,-31.8164,5.5285",
    bands=["BAND3","BAND4","CMASK"]
)

In [4]:
collection_get_data(collection)

AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250228_037_01... : Already exists
AMAZONIA_1_WFI_20250227_034_01... : Already exists
AMAZONIA_1_WFI_20250227_034_01... : Already exists
AMAZONIA_1_WFI_20250227_034_01... : Already exists
AMAZONIA_1_WFI_20250227_034_01... : Already exists
AMAZONIA_1_WFI_20250227_034_01... : Already exists
AMAZONIA_1_WFI_20250227_034_01... : Already exists
AMAZONIA_1_WFI_20250227_039_01... : Already exists
AMAZONIA_1_WFI_20250227_039_01... : Already exists
AMAZONIA_1_WFI_20250227_039_01... : Already exists
AMAZONIA_1_WFI_20250227_039_01... : Already exists
AMAZONIA_1_WFI_20250227_039_01.

In [ ]:
#from json import load

#geojson_data = load(open('br.geojson', 'r', encoding='utf-8'))

In [128]:
import rasterio
import numpy as np
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from shapely.geometry import MultiPolygon
from shapely.ops import transform
from math import cos, pi
import pyproj
from pyproj import Transformer
import shapely
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np

def create_multipolygon(polygons, crs=None):
    """
    Create a MultiPolygon from a list of Polygons with CRS support.
    
    Args:
        polygons: List of Shapely Polygon objects
        crs: Optional CRS (Coordinate Reference System)
        
    Returns:
        GeoDataFrame containing the MultiPolygon with CRS
    """
    # Create MultiPolygon
    multipoly = MultiPolygon(polygons)
    
    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=[multipoly], crs=crs)
    
    return gdf

def count_pixels_with_value(raster_path, target_value):
    """
    Counts the number of pixels in a raster that match a specific value.
    
    Args:
        raster_path (str): Path to the raster file
        target_value (int/float): The pixel value to count
        
    Returns:
        int: Count of pixels with the target value
    """
    # Open the raster file
    with rasterio.open(raster_path) as src:
        # Read all data (assuming single-band raster)
        data = src.read(1)
        
        # Count pixels with the target value
        count = (data == target_value).sum()
        
        return dict(total=data.size, count=count)
    
def get_dataset_extents(datasets):
    extents = []
    for ds in datasets:
        # Get the bounding box coordinates
        left, bottom, right, top = ds.bounds
        
        # Create a shapely Polygon representing the extent
        extent = box(left, bottom, right, top)
        
        data_proj = ds.crs
        proj_converter = Transformer.from_crs(data_proj, pyproj.CRS.from_epsg(4326), always_xy=True).transform
        reproj_bbox = transform(proj_converter, extent)
        
        # Store both the geometry and CRS
        extents.append(reproj_bbox)
        
    return MultiPolygon(extents).bounds

def merge_tifs_with_extent(tif_files, output_path, extend):

    try:
        # Merge while respecting each image's native position
        mosaic, out_transform = merge(
            tif_files,
            method='first',  
            bounds=None,     
            res=None,        
            nodata=None,     
            precision=7      
        )
        
        latitude = 0  # Example: equator (adjust for your location)
        out_height, out_width = (extend[3]-extend[1])/(64/111320),(extend[2]-extend[0])/(64/(111320*abs(cos(latitude*(pi/180)))))
        
        # Write output
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=out_height,
            width=out_width,
            count=src_files[0].count,
            dtype=mosaic.dtype,
            crs=4326,
            transform=out_transform,
            nodata=src_files[0].nodata,
            compress='lzw',  
            tiled=True       
        ) as dst:
            dst.write(mosaic)
            
        print(f"Successfully merged {len(tif_files)} files to {output_path}")
        return True
    
    except Exception as e:
        print(f"Error during merge: {str(e)}")
        return False
    
    finally:
        # Close all source files
        for src in src_files:
            src.close()
            
def local_mosaic(data_dir, collection, output_dir, start_year, start_month, start_day, duration_months, bands, mosaic_method, geom=None, grid=None, grid_id=None):
    
    #bdc_grids_data = load_json()
    #selected_tile = ''
    #for g in bdc_grids_data['grids']:
    #    if (g['name'] == grid):
    #        for tile in g['features']:
    #            if tile['properties']['tile'] == grid_id:
    #                selected_tile = tile
    #geometry = selected_tile['properties']['geometry']
    
    if (mosaic_method=='lcf'):
        data_dir = os.path.join(data_dir+'/'+collection)
       
        lcf_list = []
        
        for path in os.listdir(data_dir):
            scenes_list = []
            for file in os.listdir(os.path.join(data_dir, path, 'CMASK')):
                pixel_count = count_pixels_with_value(os.path.join(data_dir, path, 'CMASK', file), 127)
                scenes_list.append(dict(date=file.split("_")[3], clean_percentage=float(pixel_count['count']/pixel_count['total']), scene=path, file=''))
            for file in os.listdir(os.path.join(data_dir, path, bands[0])):
                next((item.update(file=os.path.join(data_dir, path, bands[0], file)) for item in scenes_list if item['date'] == file.split("_")[3]), None)
            sorted_data = sorted(scenes_list, key=lambda x: x['clean_percentage'], reverse=True)
            #print(sorted_data)
            lcf_list.append(sorted_data[0])
        
        tif_files = []
        for file in lcf_list:
            tif_files.append(file['file'])  
            
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        output_file = os.path.join(output_dir, "mosaic-amazonia-brazil-2m.tif")  
        
        datasets = [rasterio.open(file) for file in tif_files]        

        extents = get_dataset_extents(datasets)

        merge_tifs_with_extent(tif_files, output_file, extents)

In [129]:
result = local_mosaic(
    data_dir=os.path.join(path_dir),
    collection="AMZ1-WFI-L4-SR-1", 
    grid="BDC_LG_V2",
    grid_id="007004",
    #geom=geojson_data['features'][0]['geometry']
    output_dir=os.path.join(path_dir, "output"),   
    mosaic_method="lcf", 
    start_year=2025,
    start_month=1,
    start_day=1,
    duration_months=2, 
    bands=['BAND3'],
)

/home/sansigolo/.local/lib/python3.10/site-packages/rasterio/merge.py:195: RasterioDeprecationWarning: The precision parameter is unused, deprecated, and will be removed in 2.0.0.
  warnings.warn(


Successfully merged 44 files to /media/sansigolo/67f68fb6-8f0b-4be2-8fdf-a1cd72d22aad/sansigolo/simple-cube-mosaic/output/mosaic-amazonia-brazil-2m.tif
